# Understand your dataset with prototypes and criticisms

## Configuration and imports

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import pandas as pd
import numpy as np
from aix360.algorithms.protodash import ProtodashExplainer

In [3]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

## Load dataset

In [4]:
data = pd.read_feather("data/heloc/heloc_preprocessed.feather")
data.head(20).transpose()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
RiskPerformance,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0
ExternalRiskEstimate,55.0,61.0,67.0,66.0,81.0,59.0,54.0,68.0,59.0,61.0,82.0,50.0,74.0,69.0,63.0,63.0,61.0,62.0,89.0,65.0
MSinceOldestTradeOpen,144.0,58.0,66.0,169.0,333.0,137.0,88.0,148.0,324.0,79.0,96.0,311.0,274.0,256.0,262.0,184.0,256.0,131.0,244.0,43.0
MSinceMostRecentTradeOpen,4.0,15.0,5.0,1.0,27.0,11.0,7.0,7.0,2.0,4.0,5.0,25.0,2.0,6.0,26.0,3.0,19.0,4.0,9.0,2.0
AverageMInFile,84.0,41.0,24.0,73.0,132.0,78.0,37.0,65.0,138.0,36.0,47.0,76.0,66.0,117.0,95.0,77.0,68.0,59.0,147.0,26.0
NumSatisfactoryTrades,20.0,2.0,9.0,28.0,12.0,31.0,25.0,17.0,24.0,19.0,16.0,12.0,23.0,11.0,3.0,34.0,14.0,16.0,10.0,15.0
NumTrades60Ever2DerogPubRec,3.0,4.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,2.0,0.0,0.0,4.0,0.0,1.0
NumTrades90Ever2DerogPubRec,0.0,4.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,0.0,0.0,2.0,0.0,1.0
PercentTradesNeverDelq,83.0,100.0,100.0,93.0,100.0,91.0,92.0,83.0,85.0,95.0,100.0,60.0,100.0,100.0,75.0,97.0,87.0,79.0,100.0,100.0
MSinceMostRecentDelq,2.0,15.0,15.0,76.0,15.0,1.0,9.0,31.0,5.0,5.0,15.0,2.0,15.0,15.0,27.0,32.0,0.0,23.0,15.0,15.0


## Prototypes for original dataset

### Prototypes for good risk performance class

In [5]:
data_good = data[data.RiskPerformance == 0].reset_index(drop=True)

X = data_good.values

print(f"Shape of X: {X.shape}")

exp = ProtodashExplainer()
(weights, protos, obj_fn_vals) = exp.explain(X, X, m=5)

proto_df = data_good.iloc[protos]
proto_df["Weight"] = np.around(weights / np.sum(weights), 4)
proto_df.transpose()

Shape of X: (4735, 36)


/Users/felix/miniconda3/envs/ethical_ai/lib/python3.7/site-packages/cvxopt/coneprog.py:2111: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if 'x' in initvals:
/Users/felix/miniconda3/envs/ethical_ai/lib/python3.7/site-packages/cvxopt/coneprog.py:2116: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if 's' in initvals:
/Users/felix/miniconda3/envs/ethical_ai/lib/python3.7/site-packages/cvxopt/coneprog.py:2131: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if 'y' in initvals:
/Users/felix/miniconda3/envs/ethical_ai/lib/python3.7/site-packages/cvxopt/coneprog.py:2136: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if 'z' in initvals:
/Users/felix/miniconda3/envs/ethical

,849,1544,1334,132,1074
RiskPerformance,0.0000,0.0000,0.0000,0.0000,0.0000
ExternalRiskEstimate,86.0000,84.0000,79.0000,64.0000,79.0000
MSinceOldestTradeOpen,222.0000,387.0000,803.0000,88.0000,186.0000
MSinceMostRecentTradeOpen,5.0000,227.0000,3.0000,8.0000,10.0000
AverageMInFile,86.0000,322.0000,101.0000,38.0000,83.0000
NumSatisfactoryTrades,26.0000,4.0000,38.0000,3.0000,19.0000
NumTrades60Ever2DerogPubRec,0.0000,0.0000,0.0000,1.0000,1.0000
NumTrades90Ever2DerogPubRec,0.0000,0.0000,0.0000,1.0000,1.0000
PercentTradesNeverDelq,96.0000,100.0000,100.0000,75.0000,95.0000
MSinceMostRecentDelq,15.0000,15.0000,15.0000,72.0000,64.0000


### Prototypes for bad risk performance class

In [6]:
data_bad = data[data.RiskPerformance == 1].reset_index(drop=True)

X = data_bad.values

print(f"Shape of X: {X.shape}")

exp = ProtodashExplainer()
(weights, protos, obj_fn_vals) = exp.explain(X, X, m=5)

proto_df = data_bad.iloc[protos]
proto_df["Weight"] = np.around(weights / np.sum(weights), 4)
proto_df.transpose()

Shape of X: (5136, 36)


/Users/felix/miniconda3/envs/ethical_ai/lib/python3.7/site-packages/cvxopt/coneprog.py:2111: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if 'x' in initvals:
/Users/felix/miniconda3/envs/ethical_ai/lib/python3.7/site-packages/cvxopt/coneprog.py:2116: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if 's' in initvals:
/Users/felix/miniconda3/envs/ethical_ai/lib/python3.7/site-packages/cvxopt/coneprog.py:2131: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if 'y' in initvals:
/Users/felix/miniconda3/envs/ethical_ai/lib/python3.7/site-packages/cvxopt/coneprog.py:2136: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if 'z' in initvals:
/Users/felix/miniconda3/envs/ethical

,3926,3759,2865,3726,4349
RiskPerformance,1.0000,1.0000,1.0000,1.0000,1.0000
ExternalRiskEstimate,76.0000,49.0000,72.0000,74.0000,72.0000
MSinceOldestTradeOpen,212.0000,200.0000,11.0000,565.0000,383.0000
MSinceMostRecentTradeOpen,10.0000,2.0000,7.0000,4.0000,383.0000
AverageMInFile,86.0000,71.0000,9.0000,148.0000,383.0000
NumSatisfactoryTrades,28.0000,45.0000,2.0000,24.0000,1.0000
NumTrades60Ever2DerogPubRec,0.0000,0.0000,0.0000,0.0000,1.0000
NumTrades90Ever2DerogPubRec,0.0000,0.0000,0.0000,0.0000,1.0000
PercentTradesNeverDelq,100.0000,98.0000,100.0000,96.0000,100.0000
MSinceMostRecentDelq,15.0000,4.0000,15.0000,62.0000,15.0000


### Prototypes for one specific applicant

In [7]:
data_good = data[data.RiskPerformance == 0].reset_index(drop=True)

X = data_good.values[1:]
y = data_good.values[0].reshape((1, data_good.shape[1]))

print(f"Shape of X: {X.shape}")
print(f"Shape of Y: {y.shape}")

exp = ProtodashExplainer()
(weights, protos, obj_fn_vals) = exp.explain(y, X, m=5)

proto_df = data_good.iloc[protos]
proto_df["Weight"] = np.around(weights / np.sum(weights), 4)
res_df = proto_df.append(data_good.iloc[0])
res_df.transpose()

Shape of X: (4734, 36)
Shape of Y: (1, 36)


/Users/felix/miniconda3/envs/ethical_ai/lib/python3.7/site-packages/cvxopt/coneprog.py:2111: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if 'x' in initvals:
/Users/felix/miniconda3/envs/ethical_ai/lib/python3.7/site-packages/cvxopt/coneprog.py:2116: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if 's' in initvals:
/Users/felix/miniconda3/envs/ethical_ai/lib/python3.7/site-packages/cvxopt/coneprog.py:2131: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if 'y' in initvals:
/Users/felix/miniconda3/envs/ethical_ai/lib/python3.7/site-packages/cvxopt/coneprog.py:2136: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if 'z' in initvals:
/Users/felix/miniconda3/envs/ethical

,3362,1543,3206,965,569,0
RiskPerformance,0.0000,0.0,0.0000,0.000,0.0000,0.0
ExternalRiskEstimate,74.0000,61.0,76.0000,69.000,77.0000,54.0
MSinceOldestTradeOpen,195.0000,279.0,163.0000,190.000,152.0000,88.0
MSinceMostRecentTradeOpen,10.0000,4.0,7.0000,6.000,2.0000,7.0
AverageMInFile,75.0000,86.0,72.0000,66.000,67.0000,37.0
NumSatisfactoryTrades,34.0000,45.0,27.0000,30.000,12.0000,25.0
NumTrades60Ever2DerogPubRec,0.0000,0.0,0.0000,1.000,3.0000,0.0
NumTrades90Ever2DerogPubRec,0.0000,0.0,0.0000,0.000,3.0000,0.0
PercentTradesNeverDelq,100.0000,98.0,97.0000,91.000,92.0000,92.0
MSinceMostRecentDelq,15.0000,69.0,25.0000,17.000,73.0000,9.0


## Criticisms for original dataset

### Criticisms for good risk performance class

Here, we extract examples rated bad risk performance that are most similar to the examples deemed good risk performance.

In [8]:
X = data_bad.values
Y = data_good.values

print(f"Shape of X: {X.shape}")
print(f"Shape of Y: {Y.shape}")

exp = ProtodashExplainer()
(weights, protos, obj_fn_vals) = exp.explain(Y, X, m=5)

proto_df = data_bad.iloc[protos]
proto_df["Weight"] = np.around(weights / np.sum(weights), 4)
proto_df.transpose()

Shape of X: (5136, 36)
Shape of Y: (4735, 36)


/Users/felix/miniconda3/envs/ethical_ai/lib/python3.7/site-packages/cvxopt/coneprog.py:2111: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if 'x' in initvals:
/Users/felix/miniconda3/envs/ethical_ai/lib/python3.7/site-packages/cvxopt/coneprog.py:2116: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if 's' in initvals:
/Users/felix/miniconda3/envs/ethical_ai/lib/python3.7/site-packages/cvxopt/coneprog.py:2131: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if 'y' in initvals:
/Users/felix/miniconda3/envs/ethical_ai/lib/python3.7/site-packages/cvxopt/coneprog.py:2136: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if 'z' in initvals:
/Users/felix/miniconda3/envs/ethical

,2349,4349,4433,2456,1311
RiskPerformance,1.0000,1.0000,1.0000,1.0000,1.0000
ExternalRiskEstimate,80.0000,72.0000,64.0000,81.0000,88.0000
MSinceOldestTradeOpen,237.0000,383.0000,65.0000,598.0000,57.0000
MSinceMostRecentTradeOpen,0.0000,383.0000,5.0000,4.0000,43.0000
AverageMInFile,77.0000,383.0000,40.0000,205.0000,252.0000
NumSatisfactoryTrades,25.0000,1.0000,14.0000,37.0000,4.0000
NumTrades60Ever2DerogPubRec,0.0000,1.0000,1.0000,0.0000,0.0000
NumTrades90Ever2DerogPubRec,0.0000,1.0000,1.0000,0.0000,0.0000
PercentTradesNeverDelq,100.0000,100.0000,93.0000,100.0000,100.0000
MSinceMostRecentDelq,15.0000,15.0000,55.0000,15.0000,15.0000


### Criticisms for bad risk performance class

In [9]:
X = data_good.values
Y = data_bad.values

print(f"Shape of X: {X.shape}")
print(f"Shape of Y: {Y.shape}")

exp = ProtodashExplainer()
(weights, protos, obj_fn_vals) = exp.explain(Y, X, m=5)

proto_df = data_good.iloc[protos]
proto_df["Weight"] = np.around(weights / np.sum(weights), 4)
proto_df.transpose()

Shape of X: (4735, 36)
Shape of Y: (5136, 36)


/Users/felix/miniconda3/envs/ethical_ai/lib/python3.7/site-packages/cvxopt/coneprog.py:2111: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if 'x' in initvals:
/Users/felix/miniconda3/envs/ethical_ai/lib/python3.7/site-packages/cvxopt/coneprog.py:2116: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if 's' in initvals:
/Users/felix/miniconda3/envs/ethical_ai/lib/python3.7/site-packages/cvxopt/coneprog.py:2131: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if 'y' in initvals:
/Users/felix/miniconda3/envs/ethical_ai/lib/python3.7/site-packages/cvxopt/coneprog.py:2136: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if 'z' in initvals:
/Users/felix/miniconda3/envs/ethical

,101,108,2988,722,2184
RiskPerformance,0.0000,0.0000,0.0000,0.0000,0.0000
ExternalRiskEstimate,73.0000,62.0000,90.0000,83.0000,70.0000
MSinceOldestTradeOpen,219.0000,21.0000,400.0000,789.0000,214.0000
MSinceMostRecentTradeOpen,4.0000,12.0000,143.0000,6.0000,3.0000
AverageMInFile,88.0000,17.0000,249.0000,102.0000,80.0000
NumSatisfactoryTrades,23.0000,4.0000,6.0000,41.0000,52.0000
NumTrades60Ever2DerogPubRec,0.0000,0.0000,0.0000,0.0000,0.0000
NumTrades90Ever2DerogPubRec,0.0000,0.0000,0.0000,0.0000,0.0000
PercentTradesNeverDelq,100.0000,100.0000,100.0000,100.0000,96.0000
MSinceMostRecentDelq,15.0000,15.0000,15.0000,15.0000,62.0000
